# Model Deployment and Inference using SageMaker Inference Component
아래 모델 배포는 아래의 인스턴스에서 테스트 완료 되었습니다.
- [Pricing Link](https://aws.amazon.com/sagemaker/pricing/)
  

## Setting configuration 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%store -r

In [3]:
print(f"registered_model : {registered_model}")
print(f"compressed_model_path : {compressed_model_path}")

registered_model : qwen3-4b
compressed_model_path : s3://sagemaker-us-west-2-322537213286/checkpoint/Qwen/Qwen3-4B/compressed_model


## Getting inference container images



In [4]:
import sagemaker
import boto3
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

sess = sagemaker.Session()
sagemaker_client = sess.sagemaker_client
sagemaker_runtime_client = sess.sagemaker_runtime_client


print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[05/05/25 02:33:54] INFO     Found credentials from IAM Role:                                   ]8;id=866510;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=422373;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


[05/05/25 02:33:55] INFO     Found credentials from IAM Role:                                   ]8;id=600389;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=92110;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=887159;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=108389;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=810173;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=645735;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker role arn: arn:aws:iam::322537213286:role/service-role/AmazonSageMaker-ExecutionRole-20250330T160497
sagemaker session region: us-west-2


In [5]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# # retrieve the llm image uri
# llm_image = get_huggingface_llm_image_uri(
#   "huggingface",
#   session=sess,
#   version="3.0.1",
# )

llm_image="763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.6.0-tgi3.2.3-gpu-py311-cu124-ubuntu22.04-v2.0"

# print ecr image uri
print(f"llm image uri: {llm_image}")

llm image uri: 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.6.0-tgi3.2.3-gpu-py311-cu124-ubuntu22.04-v2.0


## Creating SageMaker Endpoint
- Creating EndpointConfiguration
- Creating Endpoint

### Defining the Instance type

In [6]:
instance_type = "ml.g5.2xlarge"
# # instance_type = "ml.g5.4xlarge"
# instance_type = "ml.g5.xlarge"


if instance_type == "ml.p4d.24xlarge":
    num_GPUSs = 8
elif instance_type == "ml.g5.12xlarge":
    num_GPUSs = 4
elif instance_type == "ml.g5.4xlarge":
    num_GPUSs = 1    
else:
    num_GPUSs = 1
    
print(f"{instance_type} and # of GPU {num_GPUSs} is set")

ml.g5.2xlarge and # of GPU 1 is set


### Setting the endpoint configuration

In [7]:
import time

currentTime = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
print("The current time is", currentTime)

# Set an unique endpoint config name
endpoint_config_name = f"{registered_model}-config-{currentTime}" 
print(f"Endpoint config name: {endpoint_config_name}")


# Set varient name and instance type for hosting
variant_name = "AllTraffic"
model_data_download_timeout_in_seconds = 600
container_startup_health_check_timeout_in_seconds = 600

initial_instance_count = 1
max_instance_count = 1
print(f"Initial instance count: {initial_instance_count}")
print(f"Max instance count: {max_instance_count}")

The current time is 2025-05-05-02-33-56
Endpoint config name: qwen3-4b-config-2025-05-05-02-33-56
Initial instance count: 1
Max instance count: 1


### Creating SageMaker Endpoint Configuration

In [8]:
epc_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ExecutionRoleArn=role,
    ProductionVariants=[
        {
            "VariantName": variant_name,
            "InstanceType": instance_type,
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": model_data_download_timeout_in_seconds,
            "ContainerStartupHealthCheckTimeoutInSeconds": container_startup_health_check_timeout_in_seconds,
            "ManagedInstanceScaling": {
                "Status": "ENABLED",
                "MinInstanceCount": initial_instance_count,
                "MaxInstanceCount": max_instance_count,
            },
            "RoutingConfig": {"RoutingStrategy": "LEAST_OUTSTANDING_REQUESTS"},
        }
    ]
)

### Creating Endpoint

In [9]:
import time
import logging
from botocore.exceptions import ClientError, WaiterError

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def create_and_wait_for_endpoint(sagemaker_client, sess, endpoint_name, endpoint_config_name, max_wait_time=3600, check_interval=30):
    try:
        # Create the endpoint
        ep_response = sagemaker_client.create_endpoint(
            EndpointName=endpoint_name,
            EndpointConfigName=endpoint_config_name,
        )
        logging.info(f"Creating endpoint: {endpoint_name}")
        
        # Wait for the endpoint to be created
        start_time = time.time()
        while True:
            try:
                sess.wait_for_endpoint(endpoint_name, poll=check_interval)
                logging.info(f"Endpoint {endpoint_name} is now in service")
                break
            except WaiterError as e:
                if "Max attempts exceeded" in str(e):
                    current_time = time.time()
                    if current_time - start_time > max_wait_time:
                        logging.error(f"Endpoint creation timed out after {max_wait_time} seconds")
                        raise TimeoutError(f"Endpoint creation timed out")
                    else:
                        logging.info("Endpoint is still being created. Continuing to wait...")
                else:
                    raise
        
        return ep_response
    
    except ClientError as e:
        logging.error(f"Error creating endpoint: {e}")
        raise
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        raise

In [10]:
# Usage
try:
    endpoint_name = f"{registered_model}-endpoint-{currentTime}"
    logging.info(f"Endpoint name: {endpoint_name}")
    
    ep_response = create_and_wait_for_endpoint(sagemaker_client, sess, endpoint_name, endpoint_config_name)
    logging.info("Endpoint created successfully")
except Exception as e:
    logging.error(f"Failed to create endpoint: {e}")

[05/05/25 02:33:56] INFO     Endpoint name: qwen3-4b-endpoint-2025-05-05-02-33-56                   ]8;id=593393;file:///tmp/ipykernel_88071/3371534808.py\3371534808.py]8;;\:]8;id=818671;file:///tmp/ipykernel_88071/3371534808.py#4\4]8;;\

                    INFO     Creating endpoint: qwen3-4b-endpoint-2025-05-05-02-33-56              ]8;id=55730;file:///tmp/ipykernel_88071/2110020294.py\2110020294.py]8;;\:]8;id=739132;file:///tmp/ipykernel_88071/2110020294.py#14\14]8;;\

------!

[05/05/25 02:37:27] INFO     Endpoint qwen3-4b-endpoint-2025-05-05-02-33-56 is now in service      ]8;id=941351;file:///tmp/ipykernel_88071/2110020294.py\2110020294.py]8;;\:]8;id=406351;file:///tmp/ipykernel_88071/2110020294.py#21\21]8;;\

                    INFO     Endpoint created successfully                                          ]8;id=358317;file:///tmp/ipykernel_88071/3371534808.py\3371534808.py]8;;\:]8;id=189306;file:///tmp/ipykernel_88071/3371534808.py#7\7]8;;\

## SageMaker Inference component 생성과 추론 수행

### Defining SageMaker Model

In [11]:
!aws s3 ls $compressed_model_path/finetuned/

2025-05-04 21:59:33 7315540309 model.tar.gz


In [12]:
!aws s3 ls $compressed_model_path/pretrained/

2025-05-04 22:00:37 6412145633 model.tar.gz


### 테스트 셋과 모델 설정

In [13]:
from huggingface_hub import HfFolder
from sagemaker.huggingface import HuggingFaceModel
import logging
import os
import json
import pandas as pd
from IPython.display import display, HTML

# 결과 저장 디렉토리 생성
results_dir = "inference_results"
os.makedirs(results_dir, exist_ok=True)

# 테스트할 프롬프트 목록
user_prompts = [
    '서울의 유명한 관광 코스에 대해 상세히 설명해 주세요',
    '경상남도 고성의 유명한 관광 코스에 대해 상세히 설명해 주세요',
    '전세계 미슐랭 3스타 식당의 개수는 몇 개인가요?',
    '한국의 유명 셰프에 대해 알려주세요',
    '미국의 유명 세프에 대해 알려주세요',
    '당신이 알고 있는 지식은 몇 년, 몇 월까지 인가요?',
    '야한 농담을 하게 되면 당신은 답변할 수 있나요?',
    '논어에 대해 설명해 주세요',
    '너는 남성입니다. 간호사는 일반적으로 여성이고요. 모든 간호사는 여성이라고 할 수 있나요?'
]

inference_prompt_style = """너는 reasoning, analysis, problem-solving에 advanced knowledge를 갖춘 AI Assistant입니다.
    <question> 질문에 가장 적절한 답변을 작성하세요. 최종 답변 <final>을 제시하기 전에, <question> 질문에 대해 단계별 사고 과정(chain of thoughts)을 전개하여 논리적이고 정확한 분석을 수행하세요.
    
    <question>
    {}
    </question>
    ### 주의사항:
    - 불필요한 인사말이나 서두, input은 생략하고, 바로 <response> 부터 작성해주세요.
    - 질문과 답변을 반복하지 마세요
    - 단계별 사고 과정은 충분히 상세하게 작성하되, 최종 답변은 간결하게 정리하세요
    

    
    ### 응답 형식:
    <think>
        ### THINKING
        [여기에 한국어로 단계별 사고 과정을 상세히 기술하세요. 문제를 분석하고, 가능한 접근법을 검토하며, 논리적 추론을 통해 결론에 도달하는 과정을 보여주세요.]
    </think>
    <final>
        ### FINAL-ANSWER
        [THINKING에서 도출된 결론을 간결하고 명확하게 요약하여 한국어로 최종 답변으로 제시하세요.]
    </final>

    아래 답변입니다.
    <think>
    """


# inference_prompt_style.format(question) + tokenizer.eos_token

# 모델 타입 정의
model_types = ['pretrained', 'finetuned']
model_results = {}

# SageMaker Runtime 클라이언트 생성
sagemaker_runtime = boto3.client('sagemaker-runtime')

# 공통 설정
common_config = {
    "HF_MODEL_ID": "/opt/ml/model",
    "MAX_INPUT_LENGTH": "2048",
    "MAX_TOTAL_TOKENS": "4096",
    "MAX_BATCH_PREFILL_TOKENS": "4096",
    "SM_NUM_GPUS": "1"  # 1개 GPU 사용
}

[05/05/25 02:37:28] INFO     Found credentials from IAM Role:                                   ]8;id=954940;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=122083;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

### SageMaker Inference Component 생성과 테스트 데이터의 답변 확인

In [14]:
import time
import boto3
import re
import json
import os
from IPython.display import display, HTML
import pandas as pd
import matplotlib.pyplot as plt

# SageMaker 클라이언트 설정
sagemaker_client = boto3.client('sagemaker')

model_name_list = []
# 각 모델 타입에 대해 순차적으로 처리
for model_type in model_types:
    print(f"\n\n===== PROCESSING {model_type.upper()} MODEL =====")
    
    # 모델 생성
    model_name = f"{registered_model}-{time.strftime('%Y-%m-%d-%H-%M-%S')}-{model_type}"
    ic_name = f"IC-{model_name}"  # 변수 초기화
    
    model_name_list.append(model_name) ## 삭제 시 활용
    
    print(f"Creating model: {model_name}")
    
    try:
        # HuggingFace 모델 생성
        llm_model = HuggingFaceModel(
            role=role,
            name=model_name,
            model_data=f"{compressed_model_path}/{model_type}/model.tar.gz",
            image_uri=llm_image,
            env=common_config
        )
        llm_model.create()
        print(f"Model {model_name} created successfully")

        # Inference Component 생성
        print(f"Creating inference component: {ic_name}")
        
        # 기존 Inference Component 삭제 시도
        try:
            sagemaker_client.delete_inference_component(InferenceComponentName=ic_name)
            print(f"Deleted existing inference component: {ic_name}")
        except Exception as e:
            if 'ResourceNotFoundException' in str(e):
                print(f"Inference component {ic_name} does not exist. Skipping deletion.")
            else:
                print(f"Error deleting inference component {ic_name}: {e}")
        
        # 새 Inference Component 생성
        spec = {
            "ModelName": model_name,
            "ComputeResourceRequirements": {
                "NumberOfAcceleratorDevicesRequired": 1,  # GPU 1개 사용
                "NumberOfCpuCoresRequired": 4,
                "MinMemoryRequiredInMb": 8192,
            },
        }
        
        ic_response = sagemaker_client.create_inference_component(
            InferenceComponentName=ic_name,
            EndpointName=endpoint_name,
            VariantName=variant_name,
            Specification=spec,
            RuntimeConfig={"CopyCount": 1}
        )
        print(f"Created inference component: {ic_name}")
        
        # Inference Component가 InService 상태가 될 때까지 대기
        print(f"Waiting for {ic_name} to be in service...")
        max_attempts = 30
        sleep_time = 20
        
        for attempt in range(max_attempts):
            desc = sagemaker_client.describe_inference_component(
                InferenceComponentName=ic_name
            )
            status = desc['InferenceComponentStatus']
            print(f"{ic_name}: {status}")
            
            if status == 'InService':
                print(f"{ic_name} is now in service.")
                break
            elif status == 'Failed':
                print(f"{ic_name} has failed.")
                break
            
            print(f"Attempt {attempt+1}/{max_attempts}. Waiting for {sleep_time} seconds...")
            time.sleep(sleep_time)
        
        # 모델 추론 실행 및 결과 저장
        print(f"\nRunning inference with {model_type} model...")
        model_results = {"times": [], "answers": [], "full_responses": []}
        
        for prompt_index, user_prompt in enumerate(user_prompts):
            print(f"\nTesting prompt {prompt_index + 1}: {user_prompt}")
        
            # 추론 요청 생성
            request_body = {
                "inputs": inference_prompt_style.format(user_prompt),
                "parameters": {
                    "max_new_tokens": 2048,
                    "top_p": 0.9,
                    "temperature": 0.01,
                    "use_cache": True,
                    "stop": ["<|im_end|>"]
                }
            }

            start_time = time.time()
            try:
                # 추론 요청 실행
                response = sagemaker_runtime.invoke_endpoint(
                    EndpointName=endpoint_name,
                    InferenceComponentName=ic_name,
                    ContentType='application/json',
                    Body=json.dumps(request_body)
                )
                
                # 응답 처리
                result = response['Body'].read().decode('utf-8')
                parsed_data = json.loads(result)
                answer = parsed_data[0] if isinstance(parsed_data, list) else parsed_data
                generated_text = answer['generated_text']
                
                # "아래 답변입니다." 이후의 텍스트만 추출
                answer_marker = "아래 답변입니다."
                answer_start = generated_text.find(answer_marker)
                if answer_start >= 0:
                    actual_response = generated_text[answer_start + len(answer_marker):].strip()
                else:
                    actual_response = generated_text.strip()
                
                # 응답에서 사고 과정과 최종 답변 추출
                thinking_part = ""
                final_part = ""
                
                # 태그 기반 추출 시도
                think_pattern = re.compile(r'### THINKING\s*(.*?)(?=### FINAL-ANSWER|\$)', re.DOTALL)
                final_pattern = re.compile(r'### FINAL-ANSWER\s*(.*?)\$', re.DOTALL)
                
                think_match = think_pattern.search(actual_response)
                final_match = final_pattern.search(actual_response)
                
                if think_match and final_match:
                    thinking_part = think_match.group(1).strip()
                    final_part = final_match.group(1).strip()
                    extraction_method = "keywords"
                else:
                    # 전체 텍스트 사용
                    final_part = actual_response.strip()
                    thinking_part = "사고 과정이 명확히 구분되지 않았습니다."
                    extraction_method = "full_text"
                
                elapsed_time = time.time() - start_time
                
                # 결과 저장
                result_entry = {
                    "prompt": user_prompt,
                    "thinking_process": thinking_part,
                    "final_answer": final_part,
                    "elapsed_time": round(elapsed_time, 3),
                    "extraction_method": extraction_method
                }
                
                model_results["times"].append(elapsed_time)
                model_results["answers"].append(result_entry)
                model_results["full_responses"].append(actual_response)  # 실제 응답만 저장
                
                print(f"Elapsed time: {round(elapsed_time, 3)} seconds")
                print(f"Final answer:\n{final_part}")
                
            except Exception as e:
                print(f"Error processing request: {str(e)}")
                
                model_results["times"].append(None)
                model_results["answers"].append({
                    "prompt": user_prompt,
                    "thinking_process": f"Error: {str(e)}",
                    "final_answer": f"Error: {str(e)}",
                    "elapsed_time": None,
                    "extraction_method": "error"
                })
                model_results["full_responses"].append(f"Error: {str(e)}")
        
        # 결과 파일로 저장
        result_file = f"{results_dir}/{model_type}_results.json"
        with open(result_file, "w", encoding='utf-8') as f:
            json.dump(model_results, f, indent=2, ensure_ascii=False)
        print(f"\nResults for {model_type} model saved to {result_file}")
        
    except Exception as e:
        print(f"Error processing model {model_type}: {str(e)}")
    finally:
        # 항상 Inference Component 삭제하여 리소스 정리
        if 'ic_name' in locals():  # ic_name이 정의되었는지 확인
            try:
                sagemaker_client.delete_inference_component(InferenceComponentName=ic_name)
                print(f"Deleted inference component: {ic_name}")
            except Exception as e:
                print(f"Error deleting inference component {ic_name}: {e}")



===== PROCESSING PRETRAINED MODEL =====
Creating model: qwen3-4b-2025-05-05-02-37-29-pretrained


[05/05/25 02:37:29] INFO     Creating model with name: qwen3-4b-2025-05-05-02-37-29-pretrained      ]8;id=337317;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=773683;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

Model qwen3-4b-2025-05-05-02-37-29-pretrained created successfully
Creating inference component: IC-qwen3-4b-2025-05-05-02-37-29-pretrained
Error deleting inference component IC-qwen3-4b-2025-05-05-02-37-29-pretrained: An error occurred (ValidationException) when calling the DeleteInferenceComponent operation: Could not find inference component "arn:aws:sagemaker:us-west-2:322537213286:inference-component/ic-qwen3-4b-2025-05-05-02-37-29-pretrained".
Created inference component: IC-qwen3-4b-2025-05-05-02-37-29-pretrained
Waiting for IC-qwen3-4b-2025-05-05-02-37-29-pretrained to be in service...
IC-qwen3-4b-2025-05-05-02-37-29-pretrained: Creating
Attempt 1/30. Waiting for 20 seconds...
IC-qwen3-4b-2025-05-05-02-37-29-pretrained: Creating
Attempt 2/30. Waiting for 20 seconds...
IC-qwen3-4b-2025-05-05-02-37-29-pretrained: Creating
Attempt 3/30. Waiting for 20 seconds...
IC-qwen3-4b-2025-05-05-02-37-29-pretrained: Creating
Attempt 4/30. Waiting for 20 seconds...
IC-qwen3-4b-2025-05-05-02-3

[05/05/25 02:43:19] INFO     Creating model with name: qwen3-4b-2025-05-05-02-43-18-finetuned       ]8;id=659280;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=413235;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

Model qwen3-4b-2025-05-05-02-43-18-finetuned created successfully
Creating inference component: IC-qwen3-4b-2025-05-05-02-43-18-finetuned
Error deleting inference component IC-qwen3-4b-2025-05-05-02-43-18-finetuned: An error occurred (ValidationException) when calling the DeleteInferenceComponent operation: Could not find inference component "arn:aws:sagemaker:us-west-2:322537213286:inference-component/ic-qwen3-4b-2025-05-05-02-43-18-finetuned".
Created inference component: IC-qwen3-4b-2025-05-05-02-43-18-finetuned
Waiting for IC-qwen3-4b-2025-05-05-02-43-18-finetuned to be in service...
IC-qwen3-4b-2025-05-05-02-43-18-finetuned: Creating
Attempt 1/30. Waiting for 20 seconds...
IC-qwen3-4b-2025-05-05-02-43-18-finetuned: Creating
Attempt 2/30. Waiting for 20 seconds...
IC-qwen3-4b-2025-05-05-02-43-18-finetuned: Creating
Attempt 3/30. Waiting for 20 seconds...
IC-qwen3-4b-2025-05-05-02-43-18-finetuned: Creating
Attempt 4/30. Waiting for 20 seconds...
IC-qwen3-4b-2025-05-05-02-43-18-finet

### Pretrained 모델과 Fine-tuned 모델의 최종 결과 비교

In [15]:
# 결과 비교 및 시각화
print("\n\n===== COMPARING RESULTS =====")

# 저장된 결과 파일 읽기
all_results = {}
for model_type in model_types:
    result_file = f"{results_dir}/{model_type}_results.json"
    try:
        with open(result_file, "r", encoding='utf-8') as f:
            all_results[model_type] = json.load(f)
        print(f"Loaded results for {model_type} model from {result_file}")
    except Exception as e:
        print(f"Error loading results for {model_type} model: {e}")

# 모든 질문에 대한 요약 테이블 먼저 생성
display(HTML("<h2>모든 질문에 대한 결과 요약</h2>"))

# 직접 HTML 테이블 생성
html_table = """
<table style="width:100%; border-collapse:collapse; table-layout:fixed;">
  <thead>
    <tr style="background-color:#f2f2f2;">
      <th style="width:2%; text-align:center; padding:8px; border:1px solid #ddd;">번호</th>
      <th style="width:20%; text-align:left; padding:8px; border:1px solid #ddd;">질문</th>
      <th style="width:39%; text-align:left; padding:8px; border:1px solid #ddd;">pretrained 모델 답변</th>
      <th style="width:39%; text-align:left; padding:8px; border:1px solid #ddd;">finetuned 모델 답변</th>
    </tr>
  </thead>
  <tbody>
"""

# 각 질문에 대한 모델별 답변 정리
for prompt_index, prompt in enumerate(user_prompts):
    pretrained_answer = ""
    finetuned_answer = ""
    
    # pretrained 모델 답변 추출
    if "pretrained" in all_results and prompt_index < len(all_results["pretrained"]["answers"]):
        answer = all_results["pretrained"]["answers"][prompt_index]
        final_answer = answer.get('final_answer', 'N/A')
        
        try:
            # 태그 제거 (모든 HTML 태그 형식 제거)
            final_answer = re.sub(r'<[^>]+>', '', final_answer)
            
            # "THINKING" 키워드 앞뒤로 줄바꿈 추가 (HTML 태그 사용)
            final_answer = re.sub(r'THINKING', '<strong>THINKING</strong><br><br>', final_answer)
            
            # "FINAL-ANSWER" 키워드 앞뒤로 줄바꿈 추가 (HTML 태그 사용)
            final_answer = re.sub(r'FINAL-ANSWER', '<br><br><strong>FINAL-ANSWER</strong><br><br>', final_answer)

            # 마크다운 처리
            # 1. 마크다운 헤더 제거
            final_answer = re.sub(r'#+\s+', '', final_answer)
            # 2. 마크다운 강조 표시 제거
            final_answer = re.sub(r'\*\*(.*?)\*\*', r'\1', final_answer)
            # 3. 마크다운 이탤릭체 제거
            final_answer = re.sub(r'\*(.*?)\*', r'\1', final_answer)
            # 4. 마크다운 링크 제거 - [텍스트](링크) 형식을 텍스트만 남김
            final_answer = re.sub(r'$$(.*?)$$$(.*?)$', r'\1', final_answer)
            # 5. 마크다운 목록 기호 제거
            final_answer = re.sub(r'^\s*[-*+]\s+', '', final_answer, flags=re.MULTILINE)
            # 6. 마크다운 번호 목록 제거
            final_answer = re.sub(r'^\s*\d+\.\s+', '', final_answer, flags=re.MULTILINE)
            # 7. 마크다운 코드 블록 제거
            final_answer = re.sub(r'```.*?```', '', final_answer, flags=re.DOTALL)
            # 8. 인라인 코드 제거
            final_answer = re.sub(r'`(.*?)`', r'\1', final_answer)
            
            # 줄바꿈을 공백으로 변환 (HTML 태그 제외)
            final_answer = re.sub(r'(?!<br>)\n', ' ', final_answer).strip()
            
            # 연속된 공백을 하나로 줄이기
            final_answer = re.sub(r'\s+', ' ', final_answer)
            
            pretrained_answer = final_answer
        except Exception as e:
            print(f"pretrained 모델 마크다운 처리 중 오류 발생: {e}")
            pretrained_answer = "처리 중 오류 발생"
    else:
        pretrained_answer = "결과 없음"
    
    # finetuned 모델 답변 추출
    if "finetuned" in all_results and prompt_index < len(all_results["finetuned"]["answers"]):
        answer = all_results["finetuned"]["answers"][prompt_index]
        final_answer = answer.get('final_answer', 'N/A')
        
        try:
            # 태그 제거 (모든 HTML 태그 형식 제거)
            final_answer = re.sub(r'<[^>]+>', '', final_answer)
            
            # "THINKING" 키워드 앞뒤로 줄바꿈 추가 (HTML 태그 사용)
            final_answer = re.sub(r'THINKING', '<strong>THINKING</strong><br><br>', final_answer)
            
            # "FINAL-ANSWER" 키워드 앞뒤로 줄바꿈 추가 (HTML 태그 사용)
            final_answer = re.sub(r'FINAL-ANSWER', '<br><br><strong>FINAL-ANSWER</strong><br><br>', final_answer)

            # 마크다운 처리
            # 1. 마크다운 헤더 제거
            final_answer = re.sub(r'#+\s+', '', final_answer)
            # 2. 마크다운 강조 표시 제거
            final_answer = re.sub(r'\*\*(.*?)\*\*', r'\1', final_answer)
            # 3. 마크다운 이탤릭체 제거
            final_answer = re.sub(r'\*(.*?)\*', r'\1', final_answer)
            # 4. 마크다운 링크 제거 - [텍스트](링크) 형식을 텍스트만 남김
            final_answer = re.sub(r'$$(.*?)$$$(.*?)$', r'\1', final_answer)
            # 5. 마크다운 목록 기호 제거
            final_answer = re.sub(r'^\s*[-*+]\s+', '', final_answer, flags=re.MULTILINE)
            # 6. 마크다운 번호 목록 제거
            final_answer = re.sub(r'^\s*\d+\.\s+', '', final_answer, flags=re.MULTILINE)
            # 7. 마크다운 코드 블록 제거
            final_answer = re.sub(r'```.*?```', '', final_answer, flags=re.DOTALL)
            # 8. 인라인 코드 제거
            final_answer = re.sub(r'`(.*?)`', r'\1', final_answer)
            
            # 줄바꿈을 공백으로 변환 (HTML 태그 제외)
            final_answer = re.sub(r'(?!<br>)\n', ' ', final_answer).strip()
            
            # 연속된 공백을 하나로 줄이기
            final_answer = re.sub(r'\s+', ' ', final_answer)
            
            finetuned_answer = final_answer
        except Exception as e:
            print(f"finetuned 모델 마크다운 처리 중 오류 발생: {e}")
            finetuned_answer = "처리 중 오류 발생"
    else:
        finetuned_answer = "결과 없음"
    
    # 테이블 행 추가
    html_table += f"""
    <tr>
      <td style="text-align:center; padding:8px; border:1px solid #ddd; vertical-align:top;">{prompt_index + 1}</td>
      <td style="text-align:left; padding:8px; border:1px solid #ddd; vertical-align:top; word-wrap:break-word;">{prompt[:50] + "..." if len(prompt) > 50 else prompt}</td>
      <td style="text-align:left; padding:8px; border:1px solid #ddd; vertical-align:top; word-wrap:break-word;">{pretrained_answer}</td>
      <td style="text-align:left; padding:8px; border:1px solid #ddd; vertical-align:top; word-wrap:break-word;">{finetuned_answer}</td>
    </tr>
    """

html_table += """
  </tbody>
</table>
"""

display(HTML(html_table))

# # 각 질문별 상세 결과 표시
# display(HTML("<h2>질문별 상세 결과</h2>"))

# for prompt_index, prompt in enumerate(user_prompts):
#     display(HTML(f"<h3>질문 {prompt_index + 1}</h3>"))
#     display(HTML(f"<p><b>{prompt}</b></p>"))
    
#     # 각 모델의 전체 응답 표시
#     for model_type in model_types:
#         if model_type in all_results and prompt_index < len(all_results[model_type]["full_responses"]):
#             full_response = all_results[model_type]["full_responses"][prompt_index]
            
#             try:
#                 # 태그 제거 (모든 HTML 태그 형식 제거)
#                 clean_response = re.sub(r'<[^>]+>', '', full_response)
                
#                 # "THINKING" 키워드를 강조 표시로 변환
#                 clean_response = re.sub(r'THINKING', '사고 과정:', clean_response)
                
#                 # "FINAL-ANSWER" 키워드를 강조 표시로 변환하고 줄바꿈 추가
#                 clean_response = re.sub(r'FINAL-ANSWER', '\n\n최종 답변:\n', clean_response)
                
#                 # 마크다운 강조 표시를 HTML로 변환
#                 clean_response = re.sub(r'\*\*(.*?)\*\*', r'<strong>\1</strong>', clean_response)
                
#                 # 마크다운 이탤릭체를 HTML로 변환
#                 clean_response = re.sub(r'\*(.*?)\*', r'<em>\1</em>', clean_response)
                
#                 # 연속된 빈 줄을 하나로 줄이기
#                 clean_response = re.sub(r'\n\s*\n\s*\n', '\n\n', clean_response)
                
#                 # 헤더 앞의 빈 줄 제거
#                 clean_response = re.sub(r'\n\s*\n(#+\s+|사고 과정:|최종 답변:)', r'\n\1', clean_response)
#                 clean_response = re.sub(r'^(\s*\n)+(#+\s+|사고 과정:|최종 답변:)', r'\2', clean_response)
                
#                 # 각 줄의 앞쪽 공백 제거
#                 clean_lines = []
#                 for line in clean_response.split('\n'):
#                     clean_lines.append(line.lstrip())
#                 clean_response = '\n'.join(clean_lines)
                
#             except Exception as e:
#                 print(f"전체 응답 처리 중 오류 발생: {e}")
#                 clean_response = full_response  # 오류 발생 시 원본 텍스트 사용
                
#                 # 오류가 발생해도 앞쪽 공백은 제거
#                 clean_lines = []
#                 for line in clean_response.split('\n'):
#                     clean_lines.append(line.lstrip())
#                 clean_response = '\n'.join(clean_lines)
            
#             display(HTML(f"<h4>{model_type} 모델</h4>"))
            
#             # 전체 응답 표시
#             display(HTML(f"""
#             <div style="border: 1px solid #ddd; padding: 10px; margin-bottom: 20px;">
#                 <pre style="white-space: pre-wrap; margin: 0;">{clean_response}</pre>
#             </div>
#             """))
    
#     display(HTML("<hr style='margin: 30px 0;'>"))



===== COMPARING RESULTS =====
Loaded results for pretrained model from inference_results/pretrained_results.json
Loaded results for finetuned model from inference_results/finetuned_results.json


번호,질문,pretrained 모델 답변,finetuned 모델 답변
1,서울의 유명한 관광 코스에 대해 상세히 설명해 주세요,"THINKING 질문은 서울의 유명한 관광 코스에 대한 상세한 설명을 요청합니다. 이에 대한 답변을 작성하기 위해, 먼저 서울의 주요 관광지들을 정리해야 합니다. 서울은 역사적, 문화적, 현대적인 콘텐츠가 풍부한 도시로, 관광객들이 다양한 유형의 관광지를 방문합니다. 따라서, 역사적인 장소, 현대적인 랜드마크, 문화적 명소 등을 포함한 코스를 구성해야 합니다. 또한, 각 코스의 특징, 방문 시간, 추천 활동 등을 상세히 설명해야 합니다. 이 과정에서, 관광지 간의 위치와 이동 시간, 방문 순서 등을 고려하여 논리적인 구조를 유지해야 합니다. 마지막으로, 답변은 간결하고 정보가 풍부한 형태로 정리되어야 합니다. FINAL-ANSWER 서울의 유명한 관광 코스로는 서울시청, 서울역, 동대문 시장, 경복궁, 창덕궁, 남산, 서울대공원, 롯데월드, 코엑스 등이 있습니다. 역사적인 장소로는 경복궁과 창덕궁이 있으며, 현대적인 랜드마크로는 롯데월드와 코엑스가 있습니다. 문화적 명소로는 동대문 시장과 서울시청이 있으며, 자연 경관으로는 남산과 서울대공원이 있습니다. 각 코스는 방문 시간, 특징, 추천 활동 등을 고려하여 구성되어 있으며, 관광지 간 이동 시간과 순서도 고려되어 있습니다. 이러한 코스들은 관광객들이 다양한 유형의 경험을 할 수 있도록 설계되어 있습니다.","THINKING 서울은 한국의 수도로서 다양한 관광 명소가 있습니다. 주요 관광 코스를 서울의 대표적인 관광지별로 나누어 설명해 볼게요. 인기 있는 관광지별로 서울의 관광 코스를 정리해 볼게요: 한류 온도 플라자 코스 한류 온도 플라자(LOFT)는 독특한 건축สไตล์과 다양한 레스토랑, 카페, 스포츠 시설로 유명합니다. 12월에는 서울의 첫 번째 미슐랭 3스타 식당인 '모수 서울'이 오픈할 예정이에요. LOFT에서 열리는 '한류 온도 플라자 코스'에서는 한식과 양식, 퀴진, 팝업 등 다양한 행사가 열립니다. 한류 온도 플라자에서 열리는 주요 행사: '한류 온도 플라자 퀴진'은 12월 1일부터 12월 17일까지 진행되며, 17개의 퀴진이 열립니다. '한류 온도 플라자 팝업'은 12월 1일부터 12월 17일까지 진행되며, 다양한 팝업 레스토랑이 열립니다. '한류 온도 플라자 미슐랭 3스타 축하 행사를' 12월 17일 오후 7시에 LOFT 타워에서 개최할 예정이에요. 한류 온도 플라자 코스는 한식, 양식, 그리고 다양한 팝업 레스토랑을 즐길 수 있는 이상적인 장소입니다. 12월에는 미슐랭 3스타 식당의 개업을 기념하는 축하 행사까지 열리므로, 한류 관련 이벤트를 좋아하는 관광객들에게 Highly Recommended입니다. FINAL-ANSWER 서울의 관광 코스는 다양한 명소와 볼거리를 체계적으로 정리할 수 있습니다. 주요 관광지별로 서울의 대표적인 관광 코스를 소개해 드리겠습니다: 경복궁-북촌 한옥마을-인사동 코스 경복궁: 조선시대 최초의 궁궐로, 웅장한 건축물과 아름다운 정원을 감상할 수 있습니다. 북촌 한옥마을: 전통 한옥이 잘 보존된 마을로, 한복 체험, 전통 공방, 갤러리 등을 즐길 수 있습니다. 인사동: 전통 문화의 중심지로, 골동품 상점, 전통 한복 대여점, 갤러리 등이 있습니다. 한류 온도 플라자(LOFT)-여의도 코스 한류 온도 플라자: 독특한 건축สไตล์과 다양한 레스토랑, 카페, 스포츠 시설이 있는 복합 문화공간입니다. 여의도: 금융 중심지이자 문화적 인기 지점으로, 독특한 레스토랑과 카페, 그리고 다양한 이벤트가 열립니다. 경산동-여의도-동대문디자인플라자(DDP) 코스 경산동: 전통과 현대가 어우러진 아기자기한 상점들이 있는 작은 동네입니다. 여의도: 위에서 언급한 바와 같이 금융과 문화의 중심지입니다. 동대문디자인플라자(DDP): 전 세계적으로 유명한 디자인 관련 행사와 전시가 열리는 장소입니다. 광장동-여의도-한류 온도 플라자(LOFT) 코스 광장동: 다양한 레스토랑과 카페가 있는 활기찬 동네입니다. 여의도와 한류 온도 플라자는 위에서 소개한 바와 같습니다. 이외에도 서울에는 30개의 미슐랭 3스타 식당이 있어요. 이들은 각자의 독특한 요리로 유명하며, 예약이 매우 어려울 수 있습니다. 주요 미슐랭 3스타 식당은 다음과 같아요: 모수 서울 셰프 김선 셰프 김선(동대문) 셰프 김선(광장동) 셰프 김선(여의도) 셰프 김선(경산동) 셰프 김선(북촌) 셰프 김선(인사동) 이 질문에 답하기 위해, 서울의 주요 관광 코스는 경복궁-북촌 한옥마을-인사동, 한류 온도 플라자-여의도, 그리고 경산동-여의도-DDP 등으로 정리할 수 있습니다. 또한 서울의 미슐랭 3스타 식당들을 언급하여 한류와 관련된 식재료의 고급스러움도 소개할 수 있습니다."
2,경상남도 고성의 유명한 관광 코스에 대해 상세히 설명해 주세요,"THINKING 질문은 경상남도 고성의 유명한 관광 코스에 대해 상세히 설명해 주는 요청입니다. 먼저, 고성은 경상남도의 동해안에 위치한 도시로, 자연 경관과 역사적 유적지가 풍부합니다. 이에 따라, 고성의 주요 관광 코스는 해변, 역사 유적지, 자연 경관 등 다양한 분야에서 유명합니다. 다음으로, 고성의 대표적인 관광지로는 고성해수욕장, 고성항, 고성문화마을, 고성역사, 고성도심공원 등이 있습니다. 각각의 관광지의 특징과 방문 시 추천할 수 있는 활동을 파악해야 합니다. 또한, 관광 코스의 순서나 일정을 고려할 필요가 있으며, 방문객이 가장 많이 방문하는 관광지와 그 이유를 설명해야 합니다. 마지막으로, 이러한 정보를 바탕으로 고성의 관광 코스를 체계적으로 정리하여 답변을 작성해야 합니다. FINAL-ANSWER 경상남도 고성은 동해안에 위치한 관광지로, 해변, 역사 유적지, 자연 경관이 풍부합니다. 주요 관광 코스로는 고성해수욕장, 고성항, 고성문화마을, 고성역사, 고성도심공원 등이 있습니다. 고성해수욕장은 여름에 많은 관광객이 방문하는 해변이며, 고성항은 조선 역사와 해양 문화를 느낄 수 있는 곳입니다. 고성문화마을은 전통 문화를 느낄 수 있는 곳으로, 고성역사는 지역의 역사와 문화를 이해하는 데 도움이 됩니다. 고성도심공원은 자연을 즐기기에 좋은 장소입니다. 이러한 관광지들은 자연과 역사, 문화를 모두 경험할 수 있는 곳으로, 고성의 매력을 느낄 수 있습니다.","THINKING 경상남도 고성은 공룡 화석지로 유명한 지역입니다. 이 질문에 답하기 위해서는 고성의 주요 관광지와 코스를 체계적으로 설명해야 합니다. 특히 공룡 화석지와 관련 시설, 기타 역사 문화 관광지, 그리고 현대적인 관광 명소를 균형 있게 포함해야 합니다. 안전한 산책로, 교육적인 시설, 그리고 고성의 전통 문화를 체험할 수 있는 장소도 언급하는 것이 좋을 것입니다. 각 관광지의 특징과 볼거리를 구체적으로 설명하고, 고성에서 즐길 수 있는 다양한 활동을 제시하는 것이 효과적일 것입니다. FINAL-ANSWER 경상남도 고성은 공룡 화석지로 유명한 관광지입니다. 주요 관광 코스는 다음과 같습니다: 고성 공룡 화석지 관광 코스 고성 공룡 화석지 체험관 공룡 화석의 발견 과정과 과학적 연구에 대한 교육적인 정보를 제공합니다. 다양한 공룡 모형과 복원된 화석을 감상할 수 있습니다. 안전한 산책로 이용 고성 공룡 화석지 근처에는 약 1.5km 구간으로 연결된 안전한 산책로가 있습니다. 이 산책로를 따라 화석을 직접 확인할 수 있습니다. 기타 고성 관광 명소 고성박물관 고성의 역사와 문화재를 소개하는 박물관입니다. 전통 공구, 의상, 그리고 관Local의 유물 등을 감상할 수 있습니다. 고성의 전통시장 김치, 고구마, 꿀 등 전통 음식을 맛볼 수 있는 곳입니다. 전통 공방과 제작 업장에서 직접 제작된 상품을 구매할 수 있습니다. 고성 해안 산책 남포항에서 남산까지 연결된 해안 산책로를 따라 걸을 수 있습니다. 아름다운 해안 경관을 감상할 수 있습니다. 고성에서 즐길 수 있는 활동 고성 공룡 화석지 산책: 약 1.5km 구간의 안전한 산책로를 따라 화석을 확인해 보세요. 고성박물관 방문: 고성의 역사와 문화재를深入了解하기 위해 박물관을 방문해 보세요. 전통시장 탐방:

## Deleting Resource

In [16]:
from sagemaker.predictor import Predictor

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
)

In [22]:
for model_name in model_name_list:
    try:
        inference_component_name = f"IC-{model_name}"
        print(f"Deleting inference components: [b magenta]{inference_component_name} ✅")
        
        # Delete inference component
        sagemaker_client.delete_inference_component(
            InferenceComponentName=inference_component_name
        )
    except Exception as e:
        print(f"{e}")


Deleting inference components: [b magenta]IC-qwen3-4b-2025-05-05-02-37-29-pretrained ✅
An error occurred (ValidationException) when calling the DeleteInferenceComponent operation: Could not find inference component "arn:aws:sagemaker:us-west-2:322537213286:inference-component/ic-qwen3-4b-2025-05-05-02-37-29-pretrained".
Deleting inference components: [b magenta]IC-qwen3-4b-2025-05-05-02-43-18-finetuned ✅
An error occurred (ValidationException) when calling the DeleteInferenceComponent operation: Could not find inference component "arn:aws:sagemaker:us-west-2:322537213286:inference-component/ic-qwen3-4b-2025-05-05-02-43-18-finetuned".


In [18]:
try:
    # for model_name in model_name_list:
    # print(f"Deleting model: {model_name}")
    predictor.delete_model()
except Exception as e:
    print(f"{e}")


In [19]:

try:
    print(f"Deleting endpoint: [b magenta]{predictor.endpoint_name} ✅")
    predictor.delete_endpoint()
except Exception as e:
    print(f"{e}")

print("---" * 10)
print("Done")

Deleting endpoint: [b magenta]qwen3-4b-endpoint-2025-05-05-02-33-56 ✅


[05/05/25 02:49:57] INFO     Deleting endpoint configuration with name:                             ]8;id=176624;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=142149;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4995\4995]8;;\
                             qwen3-4b-config-2025-05-05-02-33-56                                                   

                    INFO     Deleting endpoint with name: qwen3-4b-endpoint-2025-05-05-02-33-56     ]8;id=45912;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=601359;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4985\4985]8;;\

------------------------------
Done


In [20]:
# endpoint_name='qwen3-4b-endpoint-2025-05-04-09-29-33'
# ic_name = 'IC-qwen3-4b-2025-05-04-09-29-33-finetuned'

# sagemaker_client.describe_inference_component(InferenceComponentName=ic_name)
# # sagemaker_client.delete_inference_component(InferenceComponentName=ic_name)

# # 새 Inference Component 생성
# spec = {
#     "ModelName": model_name,
#     "ComputeResourceRequirements": {
#         "NumberOfAcceleratorDevicesRequired": 1,
#         "NumberOfCpuCoresRequired": 4,
#         "MinMemoryRequiredInMb": 8192,
#     },
# }

# ic_response = sagemaker_client.create_inference_component(
#     InferenceComponentName=ic_name,
#     EndpointName=endpoint_name,
#     VariantName=variant_name,
#     Specification=spec,
#     RuntimeConfig={"CopyCount": 1}
# )

# user_prompt="전세계 미슐랭 3스타 식당의 개수는 몇 개인가요?"
# # 추론 요청 생성
# request_body = {
#     "inputs": inference_prompt_style.format(user_prompt),
#     "parameters": {
#         "max_new_tokens": 2048,
#         "top_p": 0.9,
#         "temperature": 0.01,
#         "use_cache" : True,
#         "stop": ["<|im_end|>"]
#     }
# }


# # 추론 요청 실행
# response = sagemaker_runtime.invoke_endpoint(
#     EndpointName=endpoint_name,
#     InferenceComponentName=ic_name,
#     ContentType='application/json',
#     Body=json.dumps(request_body)
# )

# # 응답 처리
# result = response['Body'].read().decode('utf-8')
# parsed_data = json.loads(result)
# answer = parsed_data[0] if isinstance(parsed_data, list) else parsed_data


# print(answer['generated_text'])